Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


In [2]:
def test_angle_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)
    
    # Get harmonic angle force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    harmonic_angle_force = system.getForce(1) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom angle force and hybrid positions
    angle_force_index = 2 if is_solvated else 1
    hybrid_system = htf.hybrid_system
    custom_angle_force = hybrid_system.getForce(angle_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Remove nonbonded/exception forces
    for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 3, -1):
        hybrid_system.removeForce(i - 1)
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_angle_force.getNumGlobalParameters()):
        if custom_angle_force.getGlobalParameterName(i) == 'lambda_alchemical_angles_old':
            custom_angle_force.setGlobalParameterDefaultValue(i, lambda_old)
        if custom_angle_force.getGlobalParameterName(i) == 'lambda_alchemical_angles_new':
            custom_angle_force.setGlobalParameterDefaultValue(i, lambda_new)

    # Zero the unique old/new angles in the custom bond force
    hybrid_to_angle_indices = htf._hybrid_to_new_angle_indices if is_old else htf._hybrid_to_old_angle_indices
    for hybrid_idx, idx in hybrid_to_angle_indices.items():
        p1, p2, p3, hybrid_params = custom_angle_force.getAngleParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        index_to_zero = -3 if is_old else -1
        hybrid_params[index_to_zero] *= 0
        custom_angle_force.setAngleParameters(hybrid_idx, p1, p2, p3, hybrid_params)

    ## Get energy components of standard bond force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom bond force
    if is_solvated:
        custom_angle_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[1][1]], [components_hybrid[1][1]])
    
    print("Success! Custom angle force and standard angle force energies are equal!")
    
    if check_scale:
        
        # Get custom bond force and hybrid positions
        angle_force_index = 2 if is_solvated else 1
        hybrid_system = htf_copy.hybrid_system
        custom_angle_force = hybrid_system.getForce(angle_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        if is_solvated:
            custom_angle_force.setUsesPeriodicBoundaryConditions(True)
        
        # Remove nonbonded/exception forces
        for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 3, -1):
            hybrid_system.removeForce(i - 1)
        
        ## Get energy components of custom bond force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_angle_force.getNumGlobalParameters()):
            if custom_angle_force.getGlobalParameterName(i) == 'lambda_rest_scale_angles':
                custom_angle_force.setGlobalParameterDefaultValue(i, 0.5)
        
        ## Get energy components of custom bond force iwth scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[1][1]], [components_hybrid_scaled[1][1]])
        
        print("Success! Scaling the angle force changes the energy")

# Alanine dipeptide in vacuum

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [3]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
test_angle_energies(htf)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478274737843875), ('HarmonicAngleForce', 0.6112794918156992), ('PeriodicTorsionForce', 16.285398743794953), ('NonbondedForce', -39.44992831390246), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [9]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
test_angle_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.0655897814436777), ('HarmonicAngleForce', 5.716272534625937), ('PeriodicTorsionForce', 27.61741368984425), ('NonbondedForce', 48.30455838677404), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 5.716272534625938), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [21]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [22]:
test_angle_energies(htf, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0347836535652138), ('HarmonicAngleForce', 0.6112775462128993), ('PeriodicTorsionForce', 16.285398289983057), ('NonbondedForce', -39.449927745600284), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.38415900966915045), ('AndersenThermostat', 0.0)]
Success! Scaling the angle force changes the energy


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [13]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [14]:
test_angle_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.6315371437743451), ('HarmonicAngleForce', 33.01730546553007), ('PeriodicTorsionForce', 26.56829110407182), ('NonbondedForce', -66.90905610309893), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 33.017305465530065), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


# Alanine dipeptide in solvent

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [3]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478383746835728), ('HarmonicAngleForce', 0.6112760473448785), ('PeriodicTorsionForce', 16.285411637048487), ('NonbondedForce', -844.979569988491), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('CustomTorsionForce', 16.285403285921717), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.9897984380909909), ('HarmonicAngleForce', 6.498388077197327), ('PeriodicTorsionForce', 24.87470858998569), ('NonbondedForce', -861.1354316000933), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 6.49838807719736), ('CustomTorsionForce', 16.285403285921717), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [29]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [30]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521467), ('HarmonicAngleForce', 0.6112775462129013), ('PeriodicTorsionForce', 16.285398289983075), ('NonbondedForce', -994.6703479060853), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.38415900966915045), ('AndersenThermostat'

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [31]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [32]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.7326707819761082), ('HarmonicAngleForce', 9.40418067979313), ('PeriodicTorsionForce', 27.609566002405188), ('NonbondedForce', -1017.1329730474852), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 9.404180679793004), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.6112775462128993), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.0347836535652138), ('CustomAngleForce', 0.38415900966915045), ('AndersenThermostat', 0

# CDK2 transformation in solvent

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [34]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [35]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954754), ('HarmonicAngleForce', 29.19952360920909), ('PeriodicTorsionForce', 15.076747992617824), ('NonbondedForce', 1626.6198154499011), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('CustomAngleForce', 29.19952360920909), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [36]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [37]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899662), ('HarmonicAngleForce', 29.26297997450318), ('PeriodicTorsionForce', 15.077400397489349), ('NonbondedForce', 1633.6699025010091), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('CustomAngleForce', 29.26297997450318), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one rest region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [40]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [41]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954754), ('HarmonicAngleForce', 29.19952360920909), ('PeriodicTorsionForce', 15.076747992617824), ('NonbondedForce', 1576.6818623270603), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('CustomAngleForce', 29.19952360920909), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('CustomAngleForce', 29.19952360920909), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('CustomAngleForce', 29.176342016511466), ('AndersenThermostat', 0.0), (

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [3]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899662), ('HarmonicAngleForce', 29.16395479399548), ('PeriodicTorsionForce', 15.085562618059946), ('NonbondedForce', 1587.7415461132132), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('CustomAngleForce', 29.163954227114004), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('CustomAngleForce', 29.19952360920909), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954754), ('CustomAngleForce', 29.176342016511466), ('AndersenThermostat', 0.0), 

# 8mer in solvent

### Test one alchemical region, no rest regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.349099420288429), ('HarmonicAngleForce', 12.529213143555548), ('PeriodicTorsionForce', 157.2921232185713), ('NonbondedForce', -10442.703766657549), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288429), ('CustomAngleForce', 12.529213143555548), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [8]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 10.916305792472565), ('HarmonicAngleForce', 342.2633165915934), ('PeriodicTorsionForce', 178.0243860049997), ('NonbondedForce', 1644953.3505456133), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288429), ('CustomAngleForce', 342.2633165915935), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [9]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.349099420288429), ('HarmonicAngleForce', 12.529213143555548), ('PeriodicTorsionForce', 157.2921232185713), ('NonbondedForce', -10042.491326821702), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288429), ('CustomAngleForce', 12.529213143555548), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288429), ('CustomAngleForce', 12.529213143555548), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288429), ('CustomAngleForce', 12.468936923421134), ('AndersenThermostat', 0.0)

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [11]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [12]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 10.075114394606242), ('HarmonicAngleForce', 387.5457601922751), ('PeriodicTorsionForce', 164.82913121599913), ('NonbondedForce', 98875239.84018551), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288429), ('CustomAngleForce', 387.54576019227505), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


## Barnase:barstar (no naked charge fix)

### Test one alchemical region, no rest regions

#### Old system HarmonicAngleForce vs hybrid system CustomAngleForce

In [15]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [16]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 573.4153868946538), ('HarmonicAngleForce', 1332.8720651427552), ('PeriodicTorsionForce', 3820.9887481707037), ('NonbondedForce', 57774429.20999446), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 573.4153868946538), ('CustomAngleForce', 1332.8720651427552), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicAngleForce vs hybrid system CustomAngleForce

In [17]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [18]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 572.878245005471), ('HarmonicAngleForce', 1487.0020725409329), ('PeriodicTorsionForce', 3812.5829234616463), ('NonbondedForce', 57774666.28632938), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 573.4153868946538), ('CustomAngleForce', 1487.0020725409329), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, no rest regions

#### Old system HarmonicAngleForce vs hybrid system CustomAngleForce

In [19]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [20]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 573.4153868946538), ('HarmonicAngleForce', 1332.8720651427552), ('PeriodicTorsionForce', 3820.9887481707037), ('NonbondedForce', 57774412.957104295), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 573.4153868946538), ('CustomAngleForce', 1332.8720651427552), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 573.4153868946538), ('CustomAngleForce', 1332.8720651427552), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 573.4153868946538), ('CustomAngleForce', 1332.677038253889), ('AndersenThermostat', 0.0),

#### New system HarmonicAngleForce vs hybrid system CustomAngleForce

In [21]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [22]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 572.8700322854377), ('HarmonicAngleForce', 1481.5693931501323), ('PeriodicTorsionForce', 3812.305845329244), ('NonbondedForce', 57774677.68479212), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 573.4153868946538), ('CustomAngleForce', 1481.5693931501323), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


In [ ]:
# TO DO: actually test the scaling of the energies matches